# IMMC 2020: With testing of departments
### Importing Modules and Data

In [1]:
import pandas as pd
import copy
import re
import random
import math, statistics
import numpy as np
import progressbar
import gc
import matplotlib.pyplot as plt
from pprint import pprint

median_income = 9733/365
loss_aversion_coefficient = 2
max_shelf_capacity = 150
crowd_avoidance = 0.5

pdt_csv_data = pd.read_csv("StoreData_IMMC_CSV.csv")
print(pdt_csv_data.head())

          department product_category       product_type     brand  \
0    TV&Home Theater   TVS 30" to 45"    1080p LED Smart  Brand BB   
1  Computers&Tablets          Laptops  2-in-1 Chromebook   Brand A   
2  Computers&Tablets          Laptops  2-in-1 Chromebook  Brand BB   
3  Computers&Tablets          Laptops  2-in-1 Chromebook   Brand G   
4  Computers&Tablets          Laptops  2-in-1 Chromebook   Brand P   

                                                name  initial_price  \
0                 40" 1080p Smart LED HDTV, 5 Series         259.99   
1  2-in-1 11.6" Touch-Screen Chromebook, Intel Ce...         279.99   
2  2-in-1 12.2" Touch-Screen Chromebook, Intel Ce...         449.00   
3  2-in-1 14" Touch-Screen Chromebook, Intel Core...         549.00   
4  2-in-1 14" Touch-Screen Chromebook, Intel Core...         599.00   

   discounted_price  qty  customer_rating  brand_rating  percentage_usage  \
0            179.99    8              4.7      4.623810              0.95  

In [2]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

## Determining the Popularity of Product
### Impact of Discount on Popularity

In [3]:
# Traditional Econs Approach
def q1_over_q0(p0, p1, percentage_usage):
    x = (p0 - p1)*(percentage_usage/median_income) 
    return (math.exp(x))

# Behavioural Econs Approach
def prospect_utility(x):
    if x > 0:
        return(math.log(x+1))
    else:
        return(-loss_aversion_coefficient * math.log(-x + 1))

def increase_utility(p0, p1):
    return(prospect_utility(p0 - p1))

# Helper Variables
max_increase_utility = increase_utility(3329.99, 2199.99)
min_increase_utility = 0
max_q1_over_q0 = q1_over_q0(3329.99, 2199.99, 1)
min_q1_over_qo = 1

# Combined Effect [Between 0 and 1]
def popularity_due_to_discount(p0, p1, percentage_usage):
    traditional_econs_adjusted = (q1_over_q0(p0, p1, percentage_usage)-min_q1_over_qo)/max_q1_over_q0
    behavioural_econs_adjusted = (increase_utility(p0, p1)-min_increase_utility)/max_increase_utility

    total_adjusted = statistics.mean([traditional_econs_adjusted, behavioural_econs_adjusted])
    return(total_adjusted)

### Effect of Loss Adversion on Popularity

In [4]:
# [Between 0 and 1]
def popularity_due_to_loss_aversion(qty):
    return(math.exp(-qty/loss_aversion_coefficient))

### Effects of Saliency Bias on Popularity (TO DO)

In [5]:
def popularity_due_to_saliency_bias(size, qty):
    return sigmoid(size*qty)

### Effects of Ratings on Popularity

In [6]:
# [Between 0 and 1]
def popularity_due_to_rating(pdt_rating, brand_rating):
    raw_brand = (0.8*pdt_rating + 0.2*brand_rating)
    return(raw_brand/5)

## Creating Product Class and List

In [7]:
class product:
    # Popularity Coefficients
    pop_loss_adversion_coefficient = 0.44
    pop_saliency_coefficient = 0.6
    pop_rating_coefficient = 0.5
    
    
    # Raw Data
    def __init__(self, index, name, department, product_category, product_type, brand, initial_price, discounted_price, qty, customer_rating, brand_rating=5, percentage_usage=0.5, size=20):
        self.name = name
        self.index = index
        self.department = department
        self.product_category = product_category
        self.product_type = product_type
        self.brand = brand
        self.initial_price = initial_price
        self.discounted_price = discounted_price
        self.qty = qty
        self.customer_rating = customer_rating
        self.brand_rating = brand_rating
        self.percentage_usage = percentage_usage
        self.size = size
        
        self.popularity = 0
    
    # Processed Data
    
    def set_popularity(self):
        discount_factor = popularity_due_to_discount(self.initial_price, self.discounted_price, self.percentage_usage)
        loss_adversion_factor = popularity_due_to_loss_aversion(self.qty)
        saliency_factor = popularity_due_to_saliency_bias(self.size, self.qty)
        rating_factor = popularity_due_to_rating(self.customer_rating, self.brand_rating)
        
        initial_popularity = self.pop_loss_adversion_coefficient*loss_adversion_factor + self.pop_saliency_coefficient*saliency_factor + self.pop_rating_coefficient*rating_factor
        self.popularity = sigmoid(initial_popularity + discount_factor)
        
        
        

In [8]:
# Populating the Product List
pdt_list = []
pdt_counter = 0
for index, row in pdt_csv_data.iterrows():
    
    total_qty = row["qty"]
    unit_size = row["index_size"]
    while True:
        
        cur_qty = min(int(max_shelf_capacity/unit_size), total_qty)
        print("index", index, "department", row["department"], "qty", cur_qty, "net_size", cur_qty*unit_size)
        total_qty -= cur_qty
        
        cur_pdt = product(index, row["name"], row["department"], row["product_category"], row["product_type"], row["brand"], row["initial_price"], row["discounted_price"], cur_qty, row["customer_rating"], size=unit_size, brand_rating=row["brand_rating"], percentage_usage=row["percentage_usage"])
        cur_pdt.set_popularity()
        
        
        pdt_counter += 1
        pdt_list.append(cur_pdt)
        
        if total_qty <=0:
            break

print("pdt_list len", len(pdt_list))
print("Number of products", pdt_counter)

index 0 department TV&Home Theater qty 8 net_size 16.10151668
index 1 department Computers&Tablets qty 10 net_size 1.084217678
index 2 department Computers&Tablets qty 8 net_size 0.8673741424
index 3 department Computers&Tablets qty 10 net_size 1.084217678
index 4 department Computers&Tablets qty 8 net_size 0.8673741424
index 5 department Computers&Tablets qty 8 net_size 0.8673741424
index 6 department Computers&Tablets qty 10 net_size 1.084217678
index 7 department Computers&Tablets qty 8 net_size 0.8673741424
index 8 department Computers&Tablets qty 8 net_size 0.8673741424
index 9 department Computers&Tablets qty 10 net_size 1.084217678
index 10 department Computers&Tablets qty 10 net_size 1.084217678
index 11 department TV&Home Theater qty 10 net_size 20.12689585
index 12 department TV&Home Theater qty 12 net_size 38.310133404
index 13 department TV&Home Theater qty 12 net_size 38.310133404
index 14 department TV&Home Theater qty 12 net_size 38.310133404
index 15 department TV&Home 

In [9]:
department_popularities = {
    "Appliances": 0,
    "Audio": 0,
    "Cameras": 0,
    "Cell Phones": 0,
    "Computers&Tablets": 0,
    "TV&Home Theater": 0,
    "Video Gaming": 0
}
department_qty = {
    "Appliances": 0,
    "Audio": 0,
    "Cameras": 0,
    "Cell Phones": 0,
    "Computers&Tablets": 0,
    "TV&Home Theater": 0,
    "Video Gaming": 0
}
for pdt in pdt_list:
    department_popularities[pdt.department]+= pdt.popularity*pdt.qty
    department_qty[pdt.department]+= pdt.qty

for department, total_pop in department_popularities.items():
    print("Department:", department, "Average Popularity", total_pop/department_qty[department])

Department: Appliances Average Popularity 0.8155674215018757
Department: Audio Average Popularity 0.7490197320470443
Department: Cameras Average Popularity 0.7787335325753125
Department: Cell Phones Average Popularity 0.734969213567913
Department: Computers&Tablets Average Popularity 0.7910655224522162
Department: TV&Home Theater Average Popularity 0.7995588391995322
Department: Video Gaming Average Popularity 0.8057095230819985


In [10]:
pdt_list[4].index

4

## Creating Shelf Class and Layout Object

In [11]:
class shelf:
    # Class Variables
    
    def __init__(self, department):
        self.pdts = []
        self.pdt_set = set()
        self.department = department
        self.cur_capacity = 0
    
    def add_pdt(self, pdt):
        # Check Department
        # print("Adding Product")
        # print("  product_department", pdt.department)
        # print("  shelf_department", self.department)
        if pdt.department != self.department:
            return -1
        
        # If shelf can accomodate the product
        if self.cur_capacity + pdt.size*pdt.qty <= max_shelf_capacity:
            self.cur_capacity += pdt.size*pdt.qty
            self.pdts.append(pdt)
            self.pdt_set.add(pdt.index)
            return 0
        
        # If shelf is full
        return -1

In [12]:
class layout:
    
    def __init__(self, grid, counter, entrance, exit, shelf_list=[], pdt_list=[]):
        self.counter = counter
        self.value_of_goods_bought = 0
        self.entrance = entrance
        self.exit = exit
        
        # Grid is a 2d matrix where shelves are 1 indexed
        self.grid = copy.deepcopy(grid)
        self.shelf_list = copy.deepcopy(shelf_list)
        
        # A* Grid is a grid where shelves are labelled as 1
        self.a_star_grid = copy.deepcopy(grid)
        for i in range(len(self.a_star_grid)):
            for j in range(len(self.a_star_grid)):
                if self.a_star_grid[i][j] > 1:
                    self.a_star_grid[i][j] = 1
                    
        # Shopper Density Grid is a grid to record the density of shoppers; shelves have a density of 99
        self.shopper_density_grid = copy.deepcopy(self.a_star_grid)
        for i in range(len(self.shopper_density_grid)):
            for j in range(len(self.shopper_density_grid)):
                if self.shopper_density_grid[i][j] == 1:
                    self.shopper_density_grid[i][j] = -1
                    
        # Price Density Grid is a grid to record to value of products the customers are carrying at particular locations
        self.price_density_grid = copy.deepcopy(grid)
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                if self.price_density_grid[i][j] > 0:
                    self.price_density_grid[i][j] = 0
                    
                    
        # pdt_list contains the products that exists somewhere within the layout
        self.pdt_list = copy.deepcopy(pdt_list)
        
        
        
    # A_Star Performs a simulation of a person walking within the layout from init to goal
    # A_Star Returns a list of nodes visited on the path
    # Coordinates are written as [y,x] with [0,0] being the upper left hand corner
    def a_star(self, init, goal): 
        grid = copy.deepcopy(self.a_star_grid)
        cost = 1

        # the cost map which pushes the path closer to the goal
#         heuristic = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
#         for i in range(len(grid)):
#             for j in range(len(grid[0])):
#                 heuristic[i][j] = abs(i - goal[0]) + abs(j - goal[1])
#                 if grid[i][j] == 1:
#                     heuristic[i][j] = 99  # added extra penalty in the heuristic map

        heuristic = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                heuristic[i][j] = abs(i - goal[0]) + abs(j - goal[1])
                if grid[i][j] == 1:
                    heuristic[i][j] = 999999  # added extra penalty in the heuristic map
                else:
                    heuristic[i][j] = crowd_avoidance * self.shopper_density_grid[i][j]



        # the actions we can take
        delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right


        # function to search the path
        def search(grid, init, goal, cost, heuristic):

            closed = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the reference grid
            closed[init[0]][init[1]] = 1
            action = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the action grid

            x = init[0]
            y = init[1]
            g = 0
            f = g + heuristic[init[0]][init[0]]
            cell = [[f, g, x, y]]

            found = False  # flag that is set when search is complete
            resign = False  # flag set if we can't find expand

            while not found and not resign:
                if len(cell) == 0:
                    return "FAIL"
                else:
                    cell.sort()  # to choose the least costliest action so as to move closer to the goal
                    cell.reverse()
                    next = cell.pop()
                    x = next[2]
                    y = next[3]
                    g = next[1]

                    if x == goal[0] and y == goal[1]:
                        found = True
                    else:
                        for i in range(len(delta)):  # to try out different valid actions
                            x2 = x + delta[i][0]
                            y2 = y + delta[i][1]
                            if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                                if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                                    g2 = g + cost
                                    f2 = g2 + heuristic[x2][y2]
                                    cell.append([f2, g2, x2, y2])
                                    closed[x2][y2] = 1
                                    action[x2][y2] = i
            invpath = []
            x = goal[0]
            y = goal[1]
            invpath.append([x, y])  # we get the reverse path from here
            while x != init[0] or y != init[1]:
                x2 = x - delta[action[x][y]][0]
                y2 = y - delta[action[x][y]][1]
                x = x2
                y = y2
                invpath.append([x, y])

            path = []
            for i in range(len(invpath)):
                path.append(invpath[len(invpath) - 1 - i])
                
#             print("ACTION MAP")
#             for i in range(len(action)):
#                 print(action[i])

            return path
        return search(grid, init, goal, cost, heuristic)
    
    # Simulates Choice of Object to Buy and Deletes that Object from Object List
    def choose_pdt(self):
        #print("pdt list len", len(self.pdt_list))
        if len(self.pdt_list) == 0:
            print("RAN OUT OF ITEMS IN SHOP")
            return -1
        self.pdt_list.sort(key=lambda x: x.popularity, reverse=True)
        index = min(random.randint(0, 5), len(self.pdt_list) -1)
        chosen_pdt = copy.deepcopy(self.pdt_list[index])
        return chosen_pdt
    
    def delete_pdt(self, product_index):
        for i in range(len(self.pdt_list)):
            if self.pdt_list[i].index == product_index:
                #print("deleting pdt from list", product_index)
                
                if self.pdt_list[i].qty == 1:
                    self.pdt_list.pop(i)
                else:
                    self.pdt_list[i].qty -= 1
                    
                break
        
        return
    
    # Find Product in Sheleves
    def find_shelf(self, chosen_pdt_index):
        for shelf_index, shelf in enumerate(self.shelf_list):
                # print("Checking shelf", shelf_index)
                if chosen_pdt_index in shelf.pdt_set:
                    chosen_shelf_index = shelf_index
        return chosen_shelf_index
    
    def delete_pdt_from_shelf(self, chosen_pdt_index, shelf_index):
        # Remove product from shelf
        shelf = self.shelf_list[shelf_index]
        for i in range(len(shelf.pdts)):
            if shelf.pdts[i].index == chosen_pdt_index:
                # print("QTY of pdt", shelf.pdts[i].qty)
                    if shelf.pdts[i].qty <= 1:
                        #print("ran out of product", chosen_pdt_index)
                        del shelf.pdts[i]
                        shelf.pdt_set.discard(chosen_pdt_index)
                    else:
                        shelf.pdts[i].qty-= 1
                    break
                        
        
    def walk(self, init, goal, cur_size, cur_price):
        # Conduct A*
        delta = [(1,0), (-1,0), (0,1), (0,-1)]
        for d in delta:
            y = goal[0]+d[0]
            x = goal[1]+d[1]
            if x >= 0 and x < len(self.grid) and y >= 0 and y < len(self.grid[0]):
                if goal == self.exit:
                    [y,x] = [goal[0], goal[1]]
                path = self.a_star(init, [y,x])
                # print("init", init)
                # print("goal", [y,x])
                if path == "FAIL":
                    z=0
                    #print("FAILURE TO REACH SHELF by going to ", [y,x])
                else:
                    for cell in path:
                        #print("walked", cur_size)
                        self.shopper_density_grid[cell[0]][cell[1]] += cur_size
                        self.price_density_grid[cell[0]][cell[1]] += cur_price
                    cur_pos = [y,x]
                    #print("Visited ", y, x)
                    return cur_pos
        
    # Simulates a person entering shop
    def new_shopper(self):
        #print("--Testing New Shopper--")
        max_number_of_products = 3
        max_size = 300
        
        cur_number_of_pdt = 0
        cur_size = random.randint(40, 60)
        cur_price = 0
        
        # print("Size of Shopper:", cur_size)
        
        # Random Entrance Square
        cur_pos = random.choice(self.entrance)
        
        if len(self.pdt_list) == 0:
            return -1
        while cur_size < max_size and cur_number_of_pdt < 3 and len(self.pdt_list) > 0:
            
            # Choose what Product to Buy
            chosen_pdt = self.choose_pdt()
            if cur_size + chosen_pdt.size > max_size:
                #print("next pdt too heavy")
                break
            
            #print("Chosen Product:", chosen_pdt.index)
        
            # Find Shelf Index
            chosen_shelf_index = self.find_shelf(chosen_pdt.index)
            #print("Shelf Containing Product:", chosen_shelf_index)
        
            # Find Location of Shelf 
            for y in range(len(self.grid)):
                for x in range(len(self.grid[0])):
                    if self.grid[y][x] == chosen_shelf_index:
                        shelf_location = (y,x)
    
            # Walk from cur_pos to another shelf while tracking the movement of the shopper
            cur_pos = self.walk(cur_pos, shelf_location, cur_size, cur_price)
            self.delete_pdt(chosen_pdt.index)
            self.delete_pdt_from_shelf(chosen_pdt.index, chosen_shelf_index)
            self.value_of_goods_bought += chosen_pdt.discounted_price
            cur_number_of_pdt +=1
            cur_price += chosen_pdt.discounted_price
            cur_size += chosen_pdt.size
        
        # Walk to Counter
        cur_pos = self.walk(cur_pos, random.choice(self.counter), cur_size, cur_price)
        
        # Walk to Exit [cur_price == 0 because they aready paid]
        cur_pos = self.walk(cur_pos, random.choice(self.exit), cur_size, 0)
        #print("--Finishing Shopper--")
                
        return copy.deepcopy(self.shopper_density_grid)
        
    # Get adjusted Price Grid
    def get_price_grid(self):
        total_price = 0
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                total_price += self.price_density_grid[i][j]
        
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                self.price_density_grid[i][j] *= (self.value_of_goods_bought/total_price)
                self.price_density_grid[i][j] = int(self.price_density_grid[i][j])
        
        return copy.deepcopy(self.price_density_grid)
    
    def get_shopper_grid(self):
        return self.shopper_density_grid

## Calculating Product Damage
### Collision Damage / Self-Drops

In [13]:
def loss(shopper_density, price_density):
    p_collision = (min(shopper_density, 300)) **2 / (300*300)
    return p_collision*price_density

def total_loss(shopper_density_grid, price_density_grid):
    net_loss = 0
    for y in range(len(shopper_density_grid)):
        for x in range(len(shopper_density_grid)):
            net_loss += loss(shopper_density_grid[y][x], price_density_grid[y][x])

    return net_loss


## Simulate Layout in Figure 1
### Create and Process Layout

### Process Objects Based on Department to Place into Shelves

### Run Simulation of Shoppers

### Analysis of Crowd Density

## Simulation Cells

In [14]:
template='''
000111111111100000111111111100001111111111000001
000000000000000000000000000000000000000000000001
200000000000000000000000000000000000000000000001
200000000000000000000000000000000000000000666001
200000000000000000000000000000000000000000666001
200222003330033300400040004000400040000000666001
200222003330033300440044004400440044000000666001
200222003330033300440044004400440044000000666001
200000000000000000044004400440044004400000000001
200000000000000000044004400440044004400000000001

200222003330033300000000000000000000000000666001
200222003330033300004444400444440044444000666001
200222003330033300004444400444440044444000666001
200000000000000000000000000000000000000000666001
200000000000000000000000000000000000000000666001
200222003330033300004444400444440044444000666001
200222003330033300004444400444440044444000666001
200222003330033300000000000000000000000000666001
200000000000000000000000000000000000000000666001
200000000000000000000000000000000000000000666001

200000000000000000000000000000000000000000666001
200000000000000000000000000000000000000000666001
205505505505505500000770077777777770077000666001
205505505505505500000770077777777770077000000001
205505505505505500000000000000000000000000000001
205505505505505500000000000000000000000000666001
205505505505505500000770077777777770077000666001
205505505505505500000770077777777770077000666001
205505505505505500000000000000000000000000666001
205505505505505500000000000000000000000000666001

205505505505505500000770077777777770077000666001
205505505505505500000770077777777770077000666001
205505505505505500000000000000000000000000666001
200000000000000000000770077770077700777000666001
200000000000000000000770077770077700777000666001
200000000000000000000000000000000000000000000001
200000000000000000000000000000000000000000666001
200000000000000000000000000000000000000000666001
000000000000000000000000000000000000000000666001
000000000000000000000000000000000000000000666001

000000000000000000000000000000000000000000666001
000000000000000000000000000000000000000000666001
000000000000000000000000000000000000000000666001
000000000000000000000000000000000000000000666001
000000000000000000000000000000000000000000666001
000000000000000000000000000000000000000000666000
000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000
'''
departments = ["a","g","c","d","e","f","b"]

def generate_layout(a,g):
    if a == g:
        return ""
    index = 2
    department_used = [0,0,0,0,0,0,0,0] # Whether the number has been used
    department_used[a] =department_used[g] = 1 
    print("A:", a, "G:",g)
    layout = (template)
    layout = layout.replace(str(a), "a")
    layout = layout.replace(str(g), "g")

    for i in range(2,8):
        if not department_used[i]:
            print("replacing", i, "with", departments[index])
            layout = layout.replace(str(i), departments[index])
            department_used[i] = 1
            index += 1
    return layout
                
# for a in range(1,8):
#     for b in range(1,8):
#         print(generate_layout(a,b))

In [19]:
def split_row(word): 
    return [char for char in word] 

def evaluate_layout(test_layout):
    # Parse Layout
    test_layout = test_layout.replace('\n\n', '\n')
    test_layout = test_layout.strip("\n")

    test_layout = test_layout.split("\n")
    for i in range(len(test_layout)):
        test_layout[i] = split_row(test_layout[i])
        
    department_dictionary = {
        "a": "Appliances",
        "b": "Audio",
        "c": "Cameras",
        "d": "Cell Phones",
        "e": "Computers&Tablets",
        "f": "TV&Home Theater",
        "g": "Video Gaming",
    }
    shelf_list = [shelf("")]
    shelf_counter = 1
    for (letter, department_name) in department_dictionary.items():
        for y in range(len(test_layout)):
            for x in range(len(test_layout)):
                if str(test_layout[y][x]) == letter:
                    # print("found shelf", shelf_counter, "department:", department_name)
                    test_layout[y][x] = str(shelf_counter)
                    shelf_counter += 1
                    shelf_list.append(shelf(department_name))

    for y in range(len(test_layout)):
        for x in range(len(test_layout)):
            if not str(test_layout[y][x]).isdigit():
                test_layout[y][x] = str(shelf_counter)
                shelf_counter +=1
            test_layout[y][x] = int(test_layout[y][x])



    max_shelf_index = shelf_counter
    
    
    # Put The Objects in Shelves
    max_pdt_per_shelf = 3

    pdt_index = 0
    pdt_list.sort(key=lambda x: x.department)

    for shelf_index in range(1, max_shelf_index):
        print("\nShelf Index: ", shelf_index)
        cur_shelf = shelf_list[shelf_index]
        print("Shelf Department -- ", cur_shelf.department)

        for i in range(max_pdt_per_shelf):
            next_pdt = copy.deepcopy(pdt_list[pdt_index])
            if next_pdt.department == "Appliances" and cur_shelf.department != "Appliances":
                print("Insufficient Appliance Space")
                return 99999
            print("Products Department", next_pdt.department)
            if cur_shelf.add_pdt(next_pdt) != -1:
                # print("Added pdt id", next_pdt.index)
                pdt_index += 1
                if pdt_index == len(pdt_list):
                    break
            else:
                break
        # print("Products in shelf" , shelf_index, ":", cur_shelf.pdt_set)

        if pdt_index == len(pdt_list):
            print("--- Finished All Products with", shelf_index, "out of", max_shelf_index, "shelves --- ")
            break
            
    # Set up Object
    ## Test Layout with Model
    counters = [(40, 6),(40, 7),(40, 8),(40, 9),(40, 10), (40, 11), (40,12), (40,13)]
    # counters = [(24, 24),(24, 25),(24, 26),(24, 23),(24, 22)]
    entrances = [[47,30],[47,29],[47,28]]
    exit = [[48,35],[48,36],[48,37]]
    test_layout_object = layout(test_layout , counters, entrances, exit, shelf_list, pdt_list)
    print("Number of products in layout:", len(test_layout_object.pdt_list))
    
    
    # Do Simulation
    gc.collect()
    for i in progressbar.progressbar(range(600)):
        try:
            shopper_density_grid = test_layout_object.new_shopper()
            if shopper_density_grid == -1:
                print("Exhausted Shop with approximately", i, "shoppers")
                break
        except:
            pass
        
    price_density_grid = test_layout_object.get_price_grid()
    shopper_density_grid = test_layout_object.get_shopper_grid()
    
    return(np.std(shopper_density_grid, axis=(0,1)))

In [20]:
 evaluate_layout(generate_layout(4,3))

A: 4 G: 3
replacing 2 with c
replacing 5 with d
replacing 6 with e
replacing 7 with f

Shelf Index:  1
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  2
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  3
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  4
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  5
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  6
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  7
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  8
Shelf Department --  Appliances
Products Department Appliances
Products Department Appliances

Shelf Index:  9
Shelf Dep

N/A% (0 of 600) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

 Audio

Shelf Index:  306
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  307
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  308
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  309
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  310
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  311
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  312
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  313
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  314
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  315
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  316
Shelf Department --  Computers&Tablets
Products Department Audio

Shelf Index:  317
Shelf Department --  Computers&Tablets
Products De

100% (600 of 600) |######################| Elapsed Time: 0:00:03 Time:  0:00:03


389.18233806035215